# Work Place Concentration

In [2]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import gc
import time
import datetime
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
start_time = time.perf_counter()

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

kms_per_radian = 6371.0088
epsilon = 0.3 / kms_per_radian


coords = work[['work-lat','work-lon']].values #extracting coords to Numpy

db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

d=[0]*num_clusters
for i in range (0,len(clusters)):
    d[i]=len(clusters[i])
    
    
centermost_points = clusters.map(get_centermost_point)
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lat':lats,'lon':lons,'count':d})
#rep_points['r']=rep_points['duration']*rep_points['repeat']
t=rep_points.sort_values(by=['count'],ascending=False)
#Destination=pd.DataFrame([[VIN_List[hh][0],t.iloc[0]['lat'],t.iloc[0]['lon'],t.iloc[0]['duration'],t.iloc[0]['repeat'],
#          t.iloc[1]['lat'],t.iloc[1]['lon'],t.iloc[1]['duration'],t.iloc[1]['repeat']]], 
#           columns=['vin','home-lat','home-lon','home-duration','home-repeat',
#                   'work-lat','work-lon','work-duration','work-repeat'])
#Destination=Destination.append(la, ignore_index=True)
#Destination.to_csv(path, mode='a',index=False,header=False)
#del sample, db, cluster_labels, num_clusters, mydict
#del clusters,d,f, centermost_points, lats, lons, rep_points, t, Destination
#gc.collect()   #release memory

end_time = time.perf_counter()
#print("returned " + str(format(df.count(), ',')) + " records")
print("time elapsed: " + str(round((end_time - start_time)/60, 2)) + " minutes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time elapsed: 0.03 minutes